In [42]:
import requests
from bs4 import BeautifulSoup


In [43]:
headers = {
    'Accept': '*/*',
    'User-Agent': 'Custom'
}

url = 'https://hh.ru'
profession = 'Программист'

In [44]:
try:
    req = requests.get(url, headers={'User-Agent': 'Custom'})
    src = req.text
    soup = BeautifulSoup(src, 'lxml')
except Exception as ex:
    print(ex)
    soup = None

In [45]:
main_block = soup.find_all('div', class_='dashboard-tiles-item__body')

for b in main_block:
    print(b)
    print()

<div class="dashboard-tiles-item__body"><div class="dashboard-tiles-item__content"><div class="dashboard-tiles-item__title">Вакансии дня</div><div class="dashboard-tiles-item__salary"></div></div><div class="dashboard-tiles-item__counter"> </div></div>

<div class="dashboard-tiles-item__body"><div class="dashboard-tiles-item__content"><div class="dashboard-tiles-item__title">Компании дня</div><div class="dashboard-tiles-item__salary"></div></div><div class="dashboard-tiles-item__counter"><span>1972<!-- --> <!-- -->вакансии</span></div></div>

<div class="dashboard-tiles-item__body"><div class="dashboard-tiles-item__content"><div class="dashboard-tiles-item__title">Работа из дома</div><div class="dashboard-tiles-item__salary"></div></div><div class="dashboard-tiles-item__counter"><span>37265<!-- --> <!-- -->вакансий</span></div></div>

<div class="dashboard-tiles-item__body"><div class="dashboard-tiles-item__content"><div class="dashboard-tiles-item__title">Подработка</div><div class="d

In [47]:
for sub in main_block:
    if sub.find('div', class_='dashboard-tiles-item__title').text == profession:
        proffession_selected_block = sub
print(proffession_selected_block)

<div class="dashboard-tiles-item__body"><div class="dashboard-tiles-item__content"><div class="dashboard-tiles-item__title">Программист</div><div class="dashboard-tiles-item__salary">до <!-- -->505 000<!-- --> <!-- -->руб.</div></div><div class="dashboard-tiles-item__counter"><span>18345<!-- --> <!-- -->вакансий</span></div></div>
